У меня есть набор данных о драгоценных камнях. Посторим модель линейной регрессии.

In [1]:
#загрузка библиотек
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np

In [2]:
try:
    df = pd.read_csv('/diamonds.csv', index_col=0 ) #, dtype='str')
except:
    url = '<https://drive.google.com/drive/u/1/my-drive/diamonds.csv>'
    file_path = 'diamonds.csv'
    df = pd.read_csv(file_path, sep=',', index_col=0) #, dtype='str')
    df

In [3]:
# info
df.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 53940 entries, 0.23 to 0.75
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   cut      53940 non-null  object 
 1   color    53940 non-null  object 
 2   clarity  53940 non-null  object 
 3   depth    53940 non-null  float64
 4   table    53940 non-null  float64
 5   price    52284 non-null  float64
 6   x        32058 non-null  float64
 7   y        53940 non-null  float64
 8   z        53940 non-null  float64
dtypes: float64(6), object(3)
memory usage: 4.1+ MB


## Основные сведения:

- Большинство данных имеет тип float64 (7 из 9). Остальные имеют тип данных object.

`Обозначения полей`:
- cut качество (Fair, Good, Very Good, Premium, Ideal)
- color цвет J (худший) to D (лучший)
- clarity прозрачность (I1 (худшая), SI2, SI1, VS2, VS1, VVS2, VVS1, IF
(лучшая))
- depth общий процент глубины
- table ширина вершины ромба относительно самого широкого места.
- price цена
-  x длина в мм
- y ширина в мм
- z глубина в мм

- Количество строк : 5390, количество столбцов - 10.

In [4]:
# copy
final = df.copy(deep = True)

На основе представленных данных целью математической модели  может быть предсказание цены бриллианта (price) на основе его характеристик (цвет, чёткость, глубина, размер стола, размеры x, y, z).

Для такой задачи подойдет модель линейной регрессии.
Перед её строительством важно обработать данные:

- Обработать пропущенные значения (например, заполнить средними значениями).
- Преобразовать категориальные признаки в числовые, использовав Label encoding для преобразования признаков "cut", "color" и "clarity" в числовые.
- Нормализовать или стандартизировать числовые признаки при необходимости.

In [5]:
# подсчет пропусков
def nan_values (final):
  count_missing = final.isna().sum()
  prosent_missing = round(final.isna().sum() * 100 / len(final), 2)

  missing_value_fin = pd.DataFrame({'num_missing': count_missing,
                                    'prosent_missing': prosent_missing })
  return missing_value_fin

nan_values (final)

,num_missing,prosent_missing
cut,0,0.00
color,0,0.00
clarity,0,0.00
depth,0,0.00
table,0,0.00
price,1656,3.07
x,21882,40.57
y,0,0.00
z,0,0.00


### Вывод:

Мы видим 3% пропусков в цене и 40.57%  в длине.

In [6]:
# duplicated
final.duplicated().sum()

161

**Вывод**

Дубликатов не очень много (161 из 53940), но удалять их не стоит, так как можно предположить, что разные схожие алмазы имеют схожие характеристики и можно потерять нужное количество данных.

In [7]:
# Обработка пропущенных значений
final['price'].fillna(final['price'].mean(), inplace=True) # заполняем пропуски средним значением
final['x'].fillna(final['x'].mean(), inplace=True)

In [8]:
# Преобразование категориальных признаков
# LabelEncoder - это класс из библиотеки sklearn.preprocessing,
#который используется для преобразования категориальных или текстовых данных в числовые.
#Это необходимо, потому что большинство машинных алгоритмов предпочитают работать с числовыми данными.
# LabelEncoder использует произвольное присвоение чисел категориям, поэтому обычно используется,
#когда есть только 2 категории (для бинарной классификации).
#Если более двух категорий и порядок присвоения имеет значение, то лучше использовать One-Hot Encoding.
encoder = LabelEncoder()
final['cut'] = encoder.fit_transform(final['cut'])
final['color'] = encoder.fit_transform(final['color'])
final['clarity'] = encoder.fit_transform(final['clarity'])

# Функция fit_transform() сначала подгоняет модель к данным (то есть, определяет, какие уникальные значения существуют),
#затем преобразует эти значения в числовые.
#Результат преобразования затем сохраняется обратно в столбец 'cut'.
#Таким образом, все значения в столбце 'cut' заменяются соответствующими числовыми значениями.

In [9]:
# Предварительная обработка данных
X = final[['cut', 'color', 'clarity', 'depth', 'table', 'x', 'y', 'z']]# имена признаков
y = final['price'] # имя целевой переменной
# целевая переменная - переменная, которую необходимо объяснить или
# значения которой необходимо спрогнозировать в наборе прикладных данных

In [10]:
# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Обучение модели
model = LinearRegression()
model.fit(X_train, y_train)

# Предсказание на тестовой выборке
y_pred = model.predict(X_test)

# Оценка модели
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 1156.598727166698
Mean Squared Error: 3202861.9618613524
Root Mean Squared Error: 1789.654145878849


**Вывод:**

Это метрики, которые используются для оценки качества модели регрессии.
- Mean Absolute Error (MAE, средняя абсолютная ошибка): это среднее значение абсолютных разностей между истинными и прогнозируемыми значениями. MAE = 1156.6 означает, что в среднем наша модель ошибается на 1156.6 единиц при прогнозировании цены.
- Mean Squared Error (MSE, среднеквадратичная ошибка): это среднее значение квадратов разностей между истинными и прогнозируемыми значениями. MSE штрафует большие ошибки сильнее, чем маленькие, так как они возводятся в квадрат.
- Root Mean Squared Error (RMSE, квадратный корень из среднеквадратичной ошибки): это квадратный корень из MSE. RMSE = 1789.65 означает, что в среднем наша модель ошибается на 1789.65 единиц при прогнозировании цены.

# Финальный вывод
Эти метрики позволяют оценить, насколько хорошо модель предсказывает целевую переменную. Однако они не говорят о том, насколько хорошей или плохой является модель в абсолютном смысле - они полезны в основном для сравнения различных моделей или версий одной и той же модели.


Оценка качества модели зависит от контекста и требуемого уровня точности. Средняя абсолютная ошибка (MAE) составляет 1156.6, среднеквадратичная ошибка (MSE) - 3202861.96, а корень из среднеквадратической ошибки (RMSE) - 1789.65. Это значит, что в среднем модель ошибается на 1156.6 или 1789.65 единиц (в зависимости от того, какую метрику мы рассматриваем) при прогнозировании целевой переменной.

Чтобы понять, хороша ли модель, нужно сравнить эти значения с некоторым базовым уровнем или бенчмарком.

Если, например, целевая переменная варьируется в диапазоне от 0 до 10000, то ошибка в 1156.6 или 1789.65 единиц может быть довольно большой. Однако, если целевая переменная варьируется в диапазоне от 0 до 1000000, то эти ошибки могут быть приемлемыми.


Важно помнить, что  MSE и RMSE более чувствительны к большим ошибкам по сравнению с MAE, поскольку они возведены в квадрат перед усреднением.
Сравнение моей модели с базовым уровнем или бенчмарком - это способ определить, насколько хорошо моя модель работает в контексте конкретной задачи.


*Базовый уровень (Baseline)*: - это решение, которое я получила бы без использования какой-либо модели машинного обучения. Например, при прогнозе продаж, базовым уровнем может быть среднее значение продаж в прошлом.Можно сравнить ошибку моей модели с ошибкой базового уровня, чтобы увидеть, превосходит ли моя модель простое предсказание среднего значения.


*Бенчмарк (Benchmark)*: более сложный тип сравнения. Это может быть результат работы другой модели (например, модели, опубликованной в научной статье или результаты других участников на Kaggle). Сравнение моей модели с бенчмарком позволяет увидеть, насколько хорошо моя модель работает по сравнению с другими подходами.


При сравнении модели с базовым уровнем или бенчмарком важно использовать ту же метрику ошибки. Если базовый уровень или бенчмарк используют, например, RMSE, то и я должна использовать RMSE для сравнения.